In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader,random_split
import torch.nn.functional as F

In [111]:
transform=transforms.Compose([
    transforms.RandomCrop(32,padding=4,padding_mode='reflect'),
    transforms.RandomHorizontalFlip(p=0.5),

    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5),inplace=True)
])
test_transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5),inplace=True)
])
data=torchvision.datasets.CIFAR10(root='/content/',download=True,transform=transform)
test=torchvision.datasets.CIFAR10(root='/content/',download=True,train=False,transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


In [112]:
batchsize=64
train_dataset,val_dataset=random_split(data,[int((1-0.2)*len(data)),int(0.2*len(data))])
train_loader=DataLoader(train_dataset,batch_size=batchsize,shuffle=True,pin_memory=True)
val_loader=DataLoader(val_dataset,batch_size=batchsize,shuffle=False,pin_memory=True)

In [ ]:
test.__len__()

In [ ]:
from torchvision.utils import make_grid
def show_images(loader):
  for images,label in loader:
    fig,axis=plt.subplots(figsize=(10,10))

    axis.imshow(make_grid(images,10).permute(1,2,0))
    break

show_images(train_loader)

In [113]:
def conv_block(inchannels,outchannels,pool=False):
  layers=[nn.Conv2d(inchannels,outchannels,kernel_size=3,padding=1),
          nn.BatchNorm2d(outchannels),
          nn.ReLU(inplace=True)]
  if pool:
    layers.append(nn.MaxPool2d(2))
  return nn.Sequential(*layers ) 

In [114]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

    def forward(self, x):
        residual = x  
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual  
        out = self.relu(out)
        return out
    
class ResnetX(nn.Module):
  def __init__(self,in_channels,n_classes):
    super(ResnetX,self).__init__()
    self.conv1=conv_block(in_channels,64)
    self.conv2=conv_block(64,128,pool=True)
    self.res1=ResidualBlock(128,128)

    self.conv3=conv_block(128,256,pool=True)
    self.conv4=conv_block(256,512,pool=True)
    self.res2=ResidualBlock(512,512)

    self.classifier=nn.Sequential(nn.MaxPool2d(4),
                                  nn.Flatten(),
                                  nn.Dropout(0.2),
                                  nn.Linear(512,n_classes))
  def forward(self,x):
    out=self.conv1(x)
    out=self.conv2(out)
    out=self.res1.forward(out)
    out=self.conv3(out)
    out=self.conv4(out)
    out=self.res2.forward(out)
    return self.classifier(out)    
    
    

In [115]:
def train(model,train_loader,val_loader,epochs,max_lr,cost_fun,optimizer):
  device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model=model.to(device)
  optimizer=optimizer(model.parameters(),max_lr)
  scheduler=torch.optim.lr_scheduler.OneCycleLR(optimizer,max_lr,epochs*len(train_loader))
  lr=[]
  for epoch in range(epochs):
    model.train()
    train_losses=[]
    for images,labels in train_loader:
      images,labels=images.to(device),labels.to(device)
      outputs=model(images)
      loss=cost_fun(outputs,labels)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      scheduler.step()
      lr.append(optimizer.param_groups[0]['lr'])
      train_losses.append(loss.item())

    model.eval()
    correct=0
    total=0
    loss=0
    los=0
    for images,labels in train_loader:
      images,labels=images.to(device),labels.to(device)
      with torch.no_grad():
        outputs=model(images)
        loss=cost_fun(outputs,labels)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        los=los+loss.item()
    print(f'Test Accuracy: {100 * correct / total}%')
    print(f"Test Loss:{los/len(val_loader)}")  
    print(f"Train Loss:{sum(train_losses)/len(train_losses)}") 

In [116]:
model=ResnetX(3,10)
lr_max=1e-2
epochs=12
loss_fun=F.cross_entropy
optimizer=torch.optim.Adam

train(model,train_loader,val_loader,epochs,lr_max,loss_fun,optimizer)

Test Accuracy: 62.0575%
Test Loss:4.353961370553181
Train Loss:1.4543022686958313
Test Accuracy: 63.85%
Test Loss:4.2391480044194845
Train Loss:1.0880298866271974
Test Accuracy: 74.595%
Test Loss:2.986015895369706
Train Loss:0.8254315492630004
Test Accuracy: 82.2675%
Test Loss:2.107638322623672
Train Loss:0.6585623976230621
Test Accuracy: 81.285%
Test Loss:2.1653283911336
Train Loss:0.5712128617763519
Test Accuracy: 85.1175%
Test Loss:1.77706112114677
Train Loss:0.4934537008285522
Test Accuracy: 86.9975%
Test Loss:1.4993238459536984
Train Loss:0.42298823268413543
Test Accuracy: 89.1875%
Test Loss:1.2817705768118999
Train Loss:0.3614203188419342
Test Accuracy: 92.875%
Test Loss:0.8356080717484283
Train Loss:0.28913277081251143
Test Accuracy: 94.8225%
Test Loss:0.593208654196399
Train Loss:0.21772300960421562
Test Accuracy: 96.3775%
Test Loss:0.4452725158304356
Train Loss:0.1584201123178005
Test Accuracy: 96.495%
Test Loss:0.4177113026736458
Train Loss:0.1313098966807127


In [134]:

root_dir='/kaggle/working/test'
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.image_names = os.listdir(folder_path)

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = os.path.join(self.folder_path, self.image_names[idx])
        image = Image.open(img_name).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image,(''.join(filter(str.isdigit, self.image_names[idx] )))
    
    
test_data= CustomDataset(folder_path=root_dir,transform=test_transform)

In [135]:
test_loader=DataLoader(test_data,batch_size=64,shuffle=False)

In [139]:

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
predictions = []
image_ids = []

with torch.no_grad():
     for images, ids in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        image_ids.extend(ids)


submission_df = pd.DataFrame({'id': image_ids, 'label': predictions})




In [141]:

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
submission_df['label'] = [class_names[label] for label in submission_df['label']]

In [142]:
submission_df.head()

,id,label
0,95733,cat
1,259117,bird
2,51026,automobile
3,218288,bird
4,195766,airplane


In [143]:
submission_df.to_csv('submission_cifar10_V1.csv', index=False)

In [ ]:
from py7zr import unpack_7zarchive
import shutil
shutil.register_unpack_format('7zip', ['.7z'], unpack_7zarchive)

In [ ]:
shutil.unpack_archive('../input/cifar-10/test.7z')